In [ ]:
pip install transformers

In [2]:
import sys
sys.path.append('..')

In [3]:
from Dense_Baselines import NeuralEmbedder, NeuralSearchEngine
from helper import *
import json
import pandas as pd

In [4]:
config = load_config()
train_splits, val_splits, test_splits = load_data(config)

In [5]:
def model_score(data, model_name):
  type_correct = {
    "Specific": 0,
    #"Subjective": 0,
    "Commonsense": 0,
    #"Compound": 0,
    "Negated": 0,
    "Analogical": 0,
    "Temporal": 0}
  type_count = {
    "Specific": 0,
    #"Subjective": 0,
    "Commonsense": 0,
    #"Compound": 0,
    "Negated": 0,
    "Analogical": 0,
    "Temporal": 0}
    
  # number of correct predictions
  correct = 0

  embedder = NeuralEmbedder(model_name, model_name)

  # loop through each query
  for sample in data:
    for key in sample['query_type']:
        if sample['query_type'][key] == 1:
          type_count[key] += 1

    docs = []
    for description in sample["options"].values():
      docs.append(description)

    # create a search engine object for this query 
    engine = NeuralSearchEngine(embedder)
    # index the options into the search engine
    engine.index(docs)

    # check if model predicted the correct answer
    ## get the predicted description
    predicted_description = engine.search(sample["query"])
    ## loop through all correct options to find the predicted id
    for option in sample["options"]:
      if sample["options"][option] == predicted_description:
        predicted_id = option

    ## check if predicted id is the same as correct id
    if predicted_id == sample["answer"]:
      correct += 1
      for key in sample['query_type']:
        if sample['query_type'][key] == 1:
          type_correct[key] += 1

  return correct, len(data), type_correct, type_count 

In [6]:
def evaluate(name, model_name):
    results = []
    for i in range(len(train_splits)):
        test_data = test_splits[i]
        correct, total, type_correct, type_count = model_score(test_data, model_name)

        for key, val in type_correct.items():
            type_correct[key] = val*100/type_count[key]
        type_correct.update({"All":correct*100/total})
        results.append(type_correct)

    df = pd.DataFrame(results)
    df.to_csv(name+".csv")

In [ ]:
evaluate("BERT", "bert-base-uncased")
evaluate("TAS-B", "sebastian-hofstaetter/distilbert-dot-tas_b-b256-msmarco")